# Introduction
This project aims to compare the Neighborhoods of two cities that are Startup Capitals of their respective countries. For which we have chosen, Bangalore in India and, The Silicon Valley, San Francisco in United States of America. 
The primary objective of this project is to help Startup offices in Bangalore on deciding a location in San Francisco to open a branch office. A lot of startup look to open office in The Silicone Valley, and we wish to provide them guidance via this project, by helping them find places similar to their current head office, in Bangalore.  


# Data Sources
1.	The neighborhood data of the two cities is taken from Opencity and Wikipedia, respectively. This for which a are being shared bellow :
        1. https://opencity.in/data/bbmp-wards
        1. https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco
    The data from open city is in a CSV formal while the data from Wikipedia will be required to be Scrapped before being used. Beautiful Soup library will be used to scrap the data from the Wikipedia page. 
    Bangalore has 199 neighbourhoods, and San Francisco has 114 neighbourhoods.
1.	Geopy library to be used for geocodes. 
1.	Folium library will be utilized to represent the data on maps. 
1.	Scikit-learn will be used to utilize machine learning. 
1.	Foursquare API will be used to gather neighborhood data.


# Importing Data
Importing data is divided into 3 stages. 
   1. The first stage is getting list of neighbourhoods of the four cities from the above Wikipedia links. 
   1. The second stage is getting location of neighbourhoods. 
   1. The third stage is getting the venues in the neighbourhoods from Foursquare.

## Neighbourhood lists of Cities
We already have the links of the Wikipedia pages from which we can get the list of neighbourhoods in each city. Beautiful Soup library is used to extract the information from the wikitables in the pages. This data is stored in a pandas dataframe. Along with the neighbourhoods, the city name, the state name and the country name are stored.

## Geolocation of the Neighbourhoods
The geopy library is used to get the location data of the Neighbourhoods. Now in geopy library, Nominatim service is used. 
For leveraging the free service of Nominatim there is a restriction of 1 call per sec. If the calls are within a second, then a ‘timeout’ error is displayed. Therefore, there needs to be an one sec gap between each call. A sufficient gap of 2 seconds is provided by calling the sleep function.

### Timeout Errors
Even after providing a 2 sec gap in calls, there are timeout errors. So, to handle these errors is simple. Simply call the Nominatim service again for these locations after checking for network connectivity.

### Missing/No Coordinates
Due to spelling errors in the name of some locations, it will not resolve into coordinates. These can be rectified by using different spellings. Some locations will not resolve despite that. Then that data is procured manually searching on Google Maps.


In [1]:
import pandas as pd
import numpy as np
import requests
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from bs4 import BeautifulSoup

#### Importing Bangalore Data

Steps to follow : 

1. Download the Bangalore Neighbourhood data from the Link
1. Then upload the data into the BD2 SQL database in IBM could
1. After the Database has been updated then make a connection with this notebook to the Database
1. Now you can read the data into your SQL file 

_Voila!_

In [2]:
!pip install ipython-sql
%load_ext sql

In [3]:
%sql ibm_db_sa://mdm67023:gmzv%40m0fzls7b1g9@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

In [4]:
%%sql
select TABNAME,TABSCHEMA from SYSCAT.TABLES where TABSCHEMA='MDM67023'

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabname,tabschema
PETSALE,MDM67023
INTERNATIONAL_STUDENT_TEST_SCORES,MDM67023
EMPLOYEES,MDM67023
JOB_HISTORY,MDM67023
JOBS,MDM67023
DEPARTMENTS,MDM67023
LOCATIONS,MDM67023
INSTRUCTOR,MDM67023
CHICAGO_SOCIOECONOMIC_DATA,MDM67023
CHICAGO_CRIME_DATA,MDM67023


<p>Here I go practicing HTML here </p>
<p>   These new coders are too enthusiastic :P </p>
<p>
<ul>
<li>We want to use 'BANGALORE_NEIGHBOURHOOD'</li>
<li>I had forgotten the name :P</li>
</ul>
</p>

In [5]:
BN_sql= %sql select * from BANGALORE_NEIGHBOURHOOD
df_bang = BN_sql.DataFrame()
df_bang.head()

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


,Ward No,Ward Name,BBMP Zone Name,BBMP Division,BBMP Sub Division,Assembly constituency,MP Constituency
0,1,Kempegowda Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
1,2,Chowdeswari Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
2,3,Atturu,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
3,4,Yelahanka Satellite Town,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
4,5,Jakkuru,Yelahanka,Byatarayanapura,Byatarayanapura,Byatarayanapura,Bangalore North


Importing San Francisco Data

Steps: 

1. Request the webpage
1. Make a Beautiful-Soup Object
1. From the object search for apprioate HTML tag that will provide the data needed
1. Save the data after extracting it from the HTML tags

In [6]:
webPage = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco").text
soup = BeautifulSoup(webPage)

In [7]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in San Francisco - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsmuVQpAMNYAAjuLIHEAAADS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_San_Francisco","wgTitle":"List of neighborhoods in San Francisco","wgCurRevisionId":952030286,"wgRevisionId":952030286,"wgArticleId":1487556,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Webarchive template archiveis links","Webarchive template wayback link

In [8]:
headlines = soup.find_all(class_='mw-headline')
headlines = headlines[:-4]

In [9]:
sanfran_neighbourhood=[]
for headline in headlines:
    sanfran_neighbourhood.append(headline.text)
print(len(sanfran_neighbourhood))
sanfran_neighbourhood[0:5]

119


['Alamo Square',
 'Anza Vista',
 'Ashbury Heights',
 'Balboa Park',
 'Balboa Terrace']

In [10]:
df_sf = pd.DataFrame(sanfran_neighbourhood)

In [11]:
df_sf.head()

,0
0,Alamo Square
1,Anza Vista
2,Ashbury Heights
3,Balboa Park
4,Balboa Terrace


### Pre-processing Phase

In [12]:
df_sf.columns = ['Neighbourhood']
df_sf['City']='San Francisco'
df_sf['State']='CA'
df_sf['Country']='USA'

In [13]:
df_bang = df_bang[['Ward Name']]
df_bang.columns = ['Neighbourhood']
df_bang['City']='Bangalore'
df_bang['State']='KA'
df_bang['Country']='India'

In [14]:
print('The number of Neighbourhoods in banglore are {}'.format(df_bang.shape[0]))
df_bang.head()

The number of Neighbourhoods in banglore are 198


,Neighbourhood,City,State,Country
0,Kempegowda Ward,Bangalore,KA,India
1,Chowdeswari Ward,Bangalore,KA,India
2,Atturu,Bangalore,KA,India
3,Yelahanka Satellite Town,Bangalore,KA,India
4,Jakkuru,Bangalore,KA,India


In [15]:
print('The number of Neighbourhoods in San Francisco are {}'.format(df_sf.shape[0]))
df_sf.head()

The number of Neighbourhoods in San Francisco are 119


,Neighbourhood,City,State,Country
0,Alamo Square,San Francisco,CA,USA
1,Anza Vista,San Francisco,CA,USA
2,Ashbury Heights,San Francisco,CA,USA
3,Balboa Park,San Francisco,CA,USA
4,Balboa Terrace,San Francisco,CA,USA


In [16]:
df = pd.concat([df_bang,df_sf],ignore_index=True,sort=False)
df

,Neighbourhood,City,State,Country
0,Kempegowda Ward,Bangalore,KA,India
1,Chowdeswari Ward,Bangalore,KA,India
2,Atturu,Bangalore,KA,India
3,Yelahanka Satellite Town,Bangalore,KA,India
4,Jakkuru,Bangalore,KA,India
5,Thanisandra,Bangalore,KA,India
6,Byatarayanapura,Bangalore,KA,India
7,Kodigehalli,Bangalore,KA,India
8,Vidyaranyapura,Bangalore,KA,India
9,Dodda Bommasandra,Bangalore,KA,India


In [17]:
latitude=[]
longitude=[]

In [18]:
df.iterrows()

<generator object DataFrame.iterrows at 0x7f24938a2780>

In [19]:
for index, location in df.iterrows():
    address = location[0] +', '+ location[1] + ', ' + location[2]
    try:
        geolocator = Nominatim(user_agent='sunnypot1989@gmail.com')
        loca = geolocator.geocode(address)
    except GeocoderTimedOut as err:
        print(err)
        latitude.append('timeout')
        longitude.append('timeout')
    else:
        print(type(loca))
        if loca is not None:
            latitude.append(loca.latitude)
            longitude.append(loca.longitude)
            print('The geograpical coordinate of {} are {}, {}.'.format(address, loca.latitude, loca.longitude))
        else:
            latitude.append(None)
            longitude.append(None)
            print('None Coordinates for {}'.format(address))
        time.sleep(1.1)

<class 'geopy.location.Location'>
The geograpical coordinate of Kempegowda Ward, Bangalore, KA are 12.997696099999999, 77.53631245866424.
<class 'geopy.location.Location'>
The geograpical coordinate of Chowdeswari Ward, Bangalore, KA are 13.12045925, 77.5796178539993.
<class 'geopy.location.Location'>
The geograpical coordinate of Atturu, Bangalore, KA are 13.1004095, 77.8587249.
<class 'geopy.location.Location'>
The geograpical coordinate of Yelahanka Satellite Town, Bangalore, KA are 13.089139249999999, 77.5827165877116.
<class 'geopy.location.Location'>
The geograpical coordinate of Jakkuru, Bangalore, KA are 13.09831985, 77.62514557763072.
<class 'geopy.location.Location'>
The geograpical coordinate of Thanisandra, Bangalore, KA are 13.054713, 77.6339258.
<class 'geopy.location.Location'>
The geograpical coordinate of Byatarayanapura, Bangalore, KA are 13.062073550000001, 77.59639164365345.
<class 'geopy.location.Location'>
The geograpical coordinate of Kodigehalli, Bangalore, KA a

<p> If a location has no result from Nomatim, then we will manual search-></p>
        <p> None Coordinates for J P Park, Bangalore, KA </p> 
<p> Earlier we had 317 rows, Now lets check now many Latitute list have None</p>
<p> Earlier we had 317 rows, Now lets check now many Latitute list have given a Time out Error</p>      

In [20]:
print('The number of Location resulting in \'None\' variable is {}'.format(latitude.count(None)))
print('The number of Location resulting in a \'timeout error\'  is {}'.format(latitude.count('timeout')))

The number of Location resulting in 'None' variable is 60
The number of Location resulting in a 'timeout error'  is 0


Now we know that 60 data points are missing, mostly due to spelling errors. To Solve this I have decided to first Edit the spellings and try

In [21]:
df['Latitude'] = latitude
df['Longitude']= longitude
df.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,KA,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,KA,India,13.120459,77.579618
2,Atturu,Bangalore,KA,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,KA,India,13.089139,77.582717
4,Jakkuru,Bangalore,KA,India,13.098320,77.625146


In [22]:
df_missing = df[df['Latitude'].isnull()]

In [23]:
print(df_missing.shape)
df_missing.tail()

(60, 6)


,Neighbourhood,City,State,Country,Latitude,Longitude
308,University Mound,San Francisco,CA,USA,NaN,NaN
309,Upper Market,San Francisco,CA,USA,NaN,NaN
314,Westwood Highlands,San Francisco,CA,USA,NaN,NaN
315,Westwood Park,San Francisco,CA,USA,NaN,NaN
316,Yerba Buena,San Francisco,CA,USA,NaN,NaN


In [24]:
latitude2 = []
longitude2 = []
for index,location in df_missing.iterrows():
    address = location[0] +', '+ location[1] 
    try:
        geolocator = Nominatim(user_agent='sunnypot1989@gmail.com')
        loca = geolocator.geocode(address)
    except GeocoderTimedOut as err:
        print(err)
        latitude2.append('timeout')
        longitude2.append('timeout')
    else:
        print(type(loca))
        if loca is not None:
            latitude2.append(loca.latitude)
            longitude2.append(loca.longitude)
            print('The geograpical coordinate of {} are {}, {}.'.format(address, loca.latitude, loca.longitude))
        else:
            latitude2.append(None)
            longitude2.append(None)
            print('None Coordinates for {}'.format(address))
        time.sleep(1.1)

<class 'NoneType'>
None Coordinates for J P Park, Bangalore
<class 'NoneType'>
None Coordinates for Manorayana Palya, Bangalore
<class 'NoneType'>
None Coordinates for Marappana Palya, Bangalore
<class 'NoneType'>
None Coordinates for New Tippa Sandra, Bangalore
<class 'NoneType'>
None Coordinates for Sagayara Puram, Bangalore
<class 'NoneType'>
None Coordinates for Jaya Mahal, Bangalore
<class 'NoneType'>
None Coordinates for Raj Mahal Guttahalli, Bangalore
<class 'NoneType'>
None Coordinates for Kottegepalya, Bangalore
<class 'NoneType'>
None Coordinates for Gayithri Nagar, Bangalore
<class 'NoneType'>
None Coordinates for Halsoor, Bangalore
<class 'NoneType'>
None Coordinates for Vrisahbhavathi Nagar, Bangalore
<class 'NoneType'>
None Coordinates for Vannar Pet, Bangalore
<class 'NoneType'>
None Coordinates for Sudham Nagar, Bangalore
<class 'NoneType'>
None Coordinates for Binni Pete, Bangalore
<class 'NoneType'>
None Coordinates for Jagajivanaram Nagar, Bangalore
<class 'NoneType'

In [25]:
df_missing['Latitude']=latitude2
df_missing['Longitude']=longitude2
print(df_missing.shape)
df_missing.tail()

(60, 6)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Neighbourhood,City,State,Country,Latitude,Longitude
308,University Mound,San Francisco,CA,USA,37.723887,-122.410594
309,Upper Market,San Francisco,CA,USA,16.416733,120.593702
314,Westwood Highlands,San Francisco,CA,USA,14.286750,120.944712
315,Westwood Park,San Francisco,CA,USA,33.542939,-89.286648
316,Yerba Buena,San Francisco,CA,USA,-26.812158,-65.298171


In [26]:
for i,data in df_missing.iterrows():
    df.set_value([i],['Latitude'],data['Latitude'])
    df.set_value([i],['Longitude'],data['Longitude'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [27]:
#Testing one of the data point that was empty earlier : 
print(df.loc[316]['Latitude'] , df.loc[316]['Longitude'])

-26.8121581 -65.2981714


Adressing the timeout data : 

In [ ]:
df_test = df[df['Latitude']=='timeout']
df_test.index[0]

Solution for the timeout data :

In [ ]:
df_test = df[df['Latitude']=='timeout']
geolocator = Nominatim(user_agent='sunnypot1989@gmail.com')
address = df_test.iloc[0][0]+', '+df_test.iloc[0][1]
loca = geolocator.geocode(address)
print('The geograpical coordinate of {} are {}, {}.'.format(address, loca.latitude, loca.longitude))

In [ ]:
df.loc[df_test.index[0]]['Latitude'] = loca.latitude
df.loc[df_test.index[0]]['Longitude'] = loca.longitude

#### Still 38 data points remain empty :

In [28]:
df_missing = df[df['Latitude'].isnull()]

In [29]:
df_missing.shape

(38, 6)

In [30]:
df_missing

,Neighbourhood,City,State,Country,Latitude,Longitude
16,J P Park,Bangalore,KA,India,NaN,NaN
32,Manorayana Palya,Bangalore,KA,India,NaN,NaN
43,Marappana Palya,Bangalore,KA,India,NaN,NaN
57,New Tippa Sandra,Bangalore,KA,India,NaN,NaN
59,Sagayara Puram,Bangalore,KA,India,NaN,NaN
62,Jaya Mahal,Bangalore,KA,India,NaN,NaN
63,Raj Mahal Guttahalli,Bangalore,KA,India,NaN,NaN
72,Kottegepalya,Bangalore,KA,India,NaN,NaN
75,Gayithri Nagar,Bangalore,KA,India,NaN,NaN
89,Halsoor,Bangalore,KA,India,NaN,NaN


#### Manual Entry from google search

In [31]:
lat_long=[(12.0339,77.5500),(13.0302, 77.6041),(13.0206, 77.5516),(12.9718, 77.6552),(13.0071, 77.6171),(12.9999, 77.5951),(12.9996, 77.5838),(12.9857, 77.5117),(13.0007, 77.5576),(12.9817, 77.6284),(12.9920, 77.5248),(12.9550, 77.6200),(12.9592, 77.5868),(12.9758, 77.5602),(12.9656, 77.5507),(12.9979,77.5861),(12.9570, 77.5621),(12.9149, 77.5206),(12.9421, 77.5464),(12.9179, 77.5688),(13.1231, 80.0593),(12.9805, 77.6673),(12.9234, 77.6386),(12.9116, 77.5759),(12.9044, 77.5649),(12.9121, 77.5462),(12.8985, 77.6058),(12.9030, 77.6242),(12.8811, 77.6494),(12.8822, 77.5660),(37.7295031,-122.3945181),(37.7527,-122.4708),(37.7198,-122.4538),(37.7890,-122.4109),(37.7805,-122.4125),(37.7644,-122.4269),(37.7534, 122.4953),(37.7913505,-122.4230393)]

In [32]:
x = tuple(zip(*lat_long))

In [33]:
print(x[0])

(12.0339, 13.0302, 13.0206, 12.9718, 13.0071, 12.9999, 12.9996, 12.9857, 13.0007, 12.9817, 12.992, 12.955, 12.9592, 12.9758, 12.9656, 12.9979, 12.957, 12.9149, 12.9421, 12.9179, 13.1231, 12.9805, 12.9234, 12.9116, 12.9044, 12.9121, 12.8985, 12.903, 12.8811, 12.8822, 37.7295031, 37.7527, 37.7198, 37.789, 37.7805, 37.7644, 37.7534, 37.7913505)


In [34]:
df_missing_copy = df_missing.copy()

In [35]:
df_missing['Latitude'] = x[0]
df_missing['Longitude'] = x[1]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
df_missing.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
16,J P Park,Bangalore,KA,India,12.0339,77.5500
32,Manorayana Palya,Bangalore,KA,India,13.0302,77.6041
43,Marappana Palya,Bangalore,KA,India,13.0206,77.5516
57,New Tippa Sandra,Bangalore,KA,India,12.9718,77.6552
59,Sagayara Puram,Bangalore,KA,India,13.0071,77.6171


In [37]:
df_copy=df.copy()

In [38]:
for i,data in df_missing.iterrows():
    df.set_value([i],['Latitude'], data['Latitude'])
    df.set_value([i],['Longitude'],data['Longitude'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [39]:
#Testing one of the data point that was empty earlier : 
print(df.loc[59]['Latitude'] , df.loc[59]['Longitude'])

13.0071 77.6171


**_Voila!_**

_Finally displaying the Final Dataset before foursquare_

In [40]:
df

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,KA,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,KA,India,13.120459,77.579618
2,Atturu,Bangalore,KA,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,KA,India,13.089139,77.582717
4,Jakkuru,Bangalore,KA,India,13.098320,77.625146
5,Thanisandra,Bangalore,KA,India,13.054713,77.633926
6,Byatarayanapura,Bangalore,KA,India,13.062074,77.596392
7,Kodigehalli,Bangalore,KA,India,13.335238,77.637856
8,Vidyaranyapura,Bangalore,KA,India,13.076641,77.557732
9,Dodda Bommasandra,Bangalore,KA,India,13.056340,77.557118


In [41]:
df['Latitude'].isnull().value_counts()

False    317
Name: Latitude, dtype: int64

<p><i>Hence, We have no null values, so I am going to save the data and continue from here.</i></p>

In [43]:
# The code was removed by Watson Studio for sharing.

In [44]:
project.save_data(file_name = "DF_LL.csv",data = df.to_csv(index=False))

{'file_name': 'DF_LL.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-p77xaxowhj9d4v',
 'asset_id': 'fd4f06a0-10e1-415b-9a7a-724840afc203'}

### NOW WE CAN CONTINUE FROM HERE WITHOUT RE-RUNNING THE UPPER PART OF THE PROJECT....PHEW

In [45]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# For future refrence, This is not working and see the next cell 
df=pd.DataFrame(project.get_file("DF_LL.csv"))

In [46]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,KA,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,KA,India,13.120459,77.579618
2,Atturu,Bangalore,KA,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,KA,India,13.089139,77.582717
4,Jakkuru,Bangalore,KA,India,13.098320,77.625146


#### Connecting to foursquare

In [47]:
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium 

import json

from pandas.io.json import json_normalize

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [48]:
df_blr = df[df['State']=='KA']
df_blr.shape

(198, 6)

In [49]:
df_sf = df[df['State']=='CA']
df_sf.shape

(119, 6)

In [50]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [51]:
def getNearbyVenues(names, latitudes, longitudes, state, radius=500):
    
    venues_list=[]
    for name, lat, lng, state in zip(names, latitudes, longitudes, state):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            state,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                    'State',
                  'N\'hood Latitude', 
                  'N\'hood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
     
#     print(nearby_venues.head())
    return(nearby_venues)

In [52]:
blr_venues = getNearbyVenues(names=df_blr['Neighbourhood'],
                                   latitudes=df_blr['Latitude'],
                                   longitudes=df_blr['Longitude'],
                                    state=df_blr['State']
                                  )

In [53]:
print(blr_venues.shape)
blr_venues.head()

(1385, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kempegowda Ward,KA,12.997696,77.536312,Cafe Coffee Day,12.997306,77.540433,Café
1,Kempegowda Ward,KA,12.997696,77.536312,Liquor mart,12.996410,77.540195,Liquor Store
2,Kempegowda Ward,KA,12.997696,77.536312,Pizza Hut,12.997148,77.540446,Pizza Place
3,Kempegowda Ward,KA,12.997696,77.536312,Sambhav moorthy park,12.997491,77.537856,Park
4,Kempegowda Ward,KA,12.997696,77.536312,Canara Stores,12.995387,77.537896,Department Store


In [54]:
sf_venues = getNearbyVenues(names=df_sf['Neighbourhood'],
                                   latitudes=df_sf['Latitude'],
                                   longitudes=df_sf['Longitude'],
                                    state=df_sf['State']
                                  )

In [55]:
print(sf_venues.shape)
sf_venues.head()

(4706, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alamo Square,CA,37.77636,-122.434689,Alamo Square,37.776045,-122.434363,Park
1,Alamo Square,CA,37.77636,-122.434689,Alamo Square Dog Park,37.775878,-122.435740,Dog Run
2,Alamo Square,CA,37.77636,-122.434689,Painted Ladies,37.776120,-122.433389,Historic Site
3,Alamo Square,CA,37.77636,-122.434689,The Independent,37.775573,-122.437835,Rock Club
4,Alamo Square,CA,37.77636,-122.434689,The Mill,37.776425,-122.437970,Bakery


In [56]:
df_blr.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,KA,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,KA,India,13.120459,77.579618
2,Atturu,Bangalore,KA,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,KA,India,13.089139,77.582717
4,Jakkuru,Bangalore,KA,India,13.098320,77.625146


In [57]:
blr_grouped = blr_venues.groupby(['Neighbourhood']).count()

In [58]:
print('Missing Data from {} Locations'.format(df_blr.shape[0]-blr_grouped.shape[0]))

Missing Data from 31 Locations


In [59]:
missing_nhood=[]
for i in df_blr['Neighbourhood']:
    if i not in blr_grouped.index:
        missing_nhood.append(i)

In [60]:
df_blr2 = df_blr.set_index(['Neighbourhood'],drop=True)

In [61]:
for location in missing_nhood:
    
    latitude = df_blr2.loc[location]['Latitude']
    longitude = df_blr2.loc[location]['Longitude']
    print("Latitude = {} & Longitude = {} of Neighbourhood - {} ".format(latitude,longitude,location))

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)

            # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    print(results)

Latitude = 13.09831985 & Longitude = 77.62514557763072 of Neighbourhood - Jakkuru 
[]
Latitude = 13.3352382 & Longitude = 77.6378555 of Neighbourhood - Kodigehalli 
[]
Latitude = 13.05633975 & Longitude = 77.55711843496857 of Neighbourhood - Dodda Bommasandra 
[]
Latitude = 12.7528354 & Longitude = 77.7305926 of Neighbourhood - Shettihalli 
[]
Latitude = 13.0081486 & Longitude = 77.7837101 of Neighbourhood - Mallasandra 
[]
Latitude = 12.0339 & Longitude = 77.55 of Neighbourhood - J P Park 
[]
Latitude = 13.0187848 & Longitude = 77.6151144 of Neighbourhood - Kushal Nagar 
[]
Latitude = 12.9834173 & Longitude = 77.3918735 of Neighbourhood - Gangenahalli 
[]
Latitude = 12.8831037 & Longitude = 77.1827712 of Neighbourhood - Mattikere 
[]
Latitude = 13.1879708 & Longitude = 77.9010899 of Neighbourhood - Chokkasandra 
[]
Latitude = 13.0180855 & Longitude = 77.48593516675216 of Neighbourhood - Dodda Bidarakallu 
[]
Latitude = 13.01582965 & Longitude = 77.60311172230354 of Neighbourhood - Dev

_We have recieved null data from _Foursquare_ for the above location. We will proceed for the project purposes, and get back getting moreaccurate venue data when the project submission is over_ 

_We will observe a smiliar trent in San Francisco_

In [62]:
df_venues = pd.concat([blr_venues,sf_venues])

In [63]:
print(df_venues.shape)
df_venues.head()

(6091, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kempegowda Ward,KA,12.997696,77.536312,Cafe Coffee Day,12.997306,77.540433,Café
1,Kempegowda Ward,KA,12.997696,77.536312,Liquor mart,12.996410,77.540195,Liquor Store
2,Kempegowda Ward,KA,12.997696,77.536312,Pizza Hut,12.997148,77.540446,Pizza Place
3,Kempegowda Ward,KA,12.997696,77.536312,Sambhav moorthy park,12.997491,77.537856,Park
4,Kempegowda Ward,KA,12.997696,77.536312,Canara Stores,12.995387,77.537896,Department Store


### Functions for One-Hot Encoding & the like 
PS: based on Venue Category

In [64]:
def onehot_df(df_venues):
    df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
    df_onehot['Neighbourhood'] = df_venues['Neighbourhood'] 
    # move neighborhood column to the first column
    fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
    df_onehot = df_onehot[fixed_columns]

    return df_onehot

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
def sorted_dataframe(df_grouped):
    num_top_venues = 20

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighbourhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    df_venues_sorted = pd.DataFrame(columns=columns)
    df_venues_sorted['Neighbourhood'] = df_grouped['Neighbourhood']

    for ind in np.arange(df_grouped.shape[0]):
        df_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

    return df_venues_sorted

In [67]:
def map_city_clusters(df_merged,latitude,longitude):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
    
    rainbow = ['#ff0000', '#8000ff', '#3c68f9', '#d4dd80', '#f6be68', '#ff964f', '#2adddd', '#b2f396']
    indices = df_merged.groupby('Cluster Labels').count().sort_values(by='City', ascending=False).index.values
    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i+x+(i*x)**2 for i in range(kclusters)]
    #colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    #rainbow = [colors.rgb2hex(i) for i in colors_array]
    

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        indice = np.where(indices == (cluster))[0][0]
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[indice],
            fill=True,
            fill_color=rainbow[indice],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

#### BANGLORE CLUSTERING

In [68]:
blr_onehot = onehot_df(blr_venues)
print(blr_onehot.shape)
blr_onehot.head()

(1385, 185)


,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tibetan Restaurant,Toll Plaza,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
blr_grouped = blr_onehot.groupby('Neighbourhood').mean().reset_index()
print(blr_grouped.shape)
blr_grouped.head()

(167, 185)


,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tibetan Restaurant,Toll Plaza,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,A Narayanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Agaram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Agrahara Dasarahalli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0
4,Anjanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


##### why do I do this ?
blr_onehot_test = blr_onehot.groupby('Neighbourhood').sum().reset_index()
blr_onehot_test.head()

In [70]:
blr_venues_sorted = sorted_dataframe(blr_grouped)
blr_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,A Narayanapura,Pharmacy,Pizza Place,Fast Food Restaurant,Dessert Shop,Bus Stop,Women's Store,Eastern European Restaurant,Financial or Legal Service,Film Studio,...,Farm,Event Space,Electronics Store,Donut Shop,Dumpling Restaurant,Cosmetics Shop,Dog Run,Diner,Dim Sum Restaurant,Dhaba
1,Adugodi,Ice Cream Shop,Bus Station,Playground,Athletics & Sports,Café,Women's Store,Electronics Store,Fish & Chips Shop,Financial or Legal Service,...,Fast Food Restaurant,Farmers Market,Farm,Event Space,Dumpling Restaurant,Eastern European Restaurant,Flea Market,Donut Shop,Dog Run,Diner
2,Agaram,Burger Joint,Bus Station,Women's Store,Electronics Store,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Farmers Market,...,Event Space,Eastern European Restaurant,Flea Market,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dim Sum Restaurant,Dhaba,Dessert Shop
3,Agrahara Dasarahalli,Indian Restaurant,Vegetarian / Vegan Restaurant,Bus Station,Café,Bar,Bakery,Breakfast Spot,Bagel Shop,Women's Store,...,Fish Market,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Event Space,Farm,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,Anjanapura,Pool,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Farmers Market,Farm,...,Electronics Store,Eastern European Restaurant,Donut Shop,Cosmetics Shop,Dog Run,Diner,Dim Sum Restaurant,Dhaba,Dessert Shop,Department Store


In [71]:
#set number of clusters
kclusters = 8

blr_grouped_clustering = blr_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(blr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

167

**Note** : _df_blr is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [72]:
df_blr_labels = df_blr.sort_values(['Neighbourhood'])
x=[]
for i in df_blr_labels['Neighbourhood']:
    if i in blr_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_blr_labels=df_blr_labels[x]
print(df_blr_labels.shape)

(167, 6)


In [73]:
df_blr_labels['Cluster Labels'] = kmeans1.labels_
df_blr_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
55,A Narayanapura,Bangalore,KA,India,12.993195,77.670377,7
146,Adugodi,Bangalore,KA,India,12.942847,77.610416,1
113,Agaram,Bangalore,KA,India,12.945704,77.637886,1
104,Agrahara Dasarahalli,Bangalore,KA,India,12.979063,77.540682,2
195,Anjanapura,Bangalore,KA,India,12.858081,77.558071,1


In [74]:
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [75]:
map_city_clusters(df_blr_labels,latitude,longitude)

In [96]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==0,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
5,Thanisandra,0


In [97]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==1,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
146,Adugodi,1
113,Agaram,1
195,Anjanapura,1
34,Aramane Nagara,1
13,Bagalakunte,1


In [98]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==2,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
104,Agrahara Dasarahalli,2
2,Atturu,2
175,BTM Layout,2
26,Banasavadi,2
133,Bapuji Nagar,2


In [99]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==3,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
53,Hudi,3


In [100]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==4,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
192,Arakere,4
72,Kottegepalya,4
124,Marenahalli,4
47,Muneshwara Nagar,4
134,Padarayanapura,4


In [101]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==5,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
100,Kamakshipalya,5
106,Shiva Nagar,5


In [102]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==6,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
1,Chowdeswari Ward,6
188,Honga Sandra,6
41,Lakshmi Devi Nagar,6
91,Shivaji Nagar,6
148,Varthur,6


In [103]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==7,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
55,A Narayanapura,7
140,Azad Nagar,7
99,Basaveshwara Nagar,7
149,Bellanduru,7
139,Chamraja Pet,7


#### SAN FRANCISCO CLUSTERING

In [76]:
sf_onehot = onehot_df(sf_venues)
print(sf_onehot.shape)
sf_onehot.head()

(4706, 349)


,Neighbourhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Vineyard,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
sf_grouped=sf_onehot.groupby('Neighbourhood').mean().reset_index()
print(sf_grouped.shape)
sf_grouped.head()

(111, 349)


,Neighbourhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Vineyard,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alamo Square,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013514,0.013514,...,0.000000,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.013514
1,Anza Vista,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,Ashbury Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,Balboa Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.055556,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Balboa Terrace,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [78]:
sf_venues_sorted = sorted_dataframe(sf_grouped)
sf_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Alamo Square,Bar,Liquor Store,Park,BBQ Joint,Pizza Place,Record Shop,Ethiopian Restaurant,Sushi Restaurant,Hotel,...,Seafood Restaurant,Wine Bar,Roller Rink,Southern / Soul Food Restaurant,Pakistani Restaurant,Souvlaki Shop,Dog Run,Diner,Pet Store,Convenience Store
1,Anza Vista,Café,Health & Beauty Service,Donut Shop,Grocery Store,Sandwich Place,Big Box Store,Coffee Shop,Liquor Store,Tunnel,...,Mexican Restaurant,Juice Bar,Southern / Soul Food Restaurant,Bus Line,Burger Joint,Cosmetics Shop,Neighborhood,Fast Food Restaurant,Filipino Restaurant,Flea Market
2,Ashbury Heights,Thai Restaurant,Yoga Studio,Garden Center,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,...,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store
3,Balboa Park,Baseball Field,Light Rail Station,Breakfast Spot,Vietnamese Restaurant,College Gym,Café,BBQ Joint,Food Truck,Asian Restaurant,...,Gym Pool,Dessert Shop,Burger Joint,Filipino Restaurant,Food,Flea Market,Flower Shop,Food Court,Food & Drink Shop,Farmers Market
4,Balboa Terrace,Fried Chicken Joint,Dentist's Office,Football Stadium,Scenic Lookout,Eye Doctor,Park,Yoga Studio,Flower Shop,Farm,...,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food Court,Food,Food & Drink Shop,Food Stand,Food Truck,Fountain,French Restaurant


In [79]:
#set number of clusters
kclusters = 8

sf_grouped_clustering = sf_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

111

**Note** : _df_sf is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [80]:
df_sf.shape

(119, 6)

In [81]:
df_sf_labels = df_sf.sort_values(['Neighbourhood'])
x=[]
for i in df_sf_labels['Neighbourhood']:
    if i in sf_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_sf_labels=df_sf_labels[x]
print(df_sf_labels.shape)

(111, 6)


In [82]:
df_sf_labels['Cluster Labels'] = kmeans1.labels_
df_sf_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
198,Alamo Square,San Francisco,CA,USA,37.776360,-122.434689,2
199,Anza Vista,San Francisco,CA,USA,37.780836,-122.443149,2
200,Ashbury Heights,San Francisco,CA,USA,33.767144,-84.274288,6
201,Balboa Park,San Francisco,CA,USA,37.721427,-122.447547,2
202,Balboa Terrace,San Francisco,CA,USA,32.809471,-117.208557,0


In [83]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [84]:
map_city_clusters(df_sf_labels,latitude,longitude)

In [104]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==0,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
202,Balboa Terrace,0
216,Corona Heights,0
218,Crocker-Amazon,0
231,Forest Hill,0
234,Golden Gate Heights,0


In [105]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==1,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
214,Clarendon Heights,1
251,Laurel Heights,1
270,Monterey Heights,1


In [106]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==2,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
198,Alamo Square,2
199,Anza Vista,2
201,Balboa Park,2
203,Bayview,2
204,Belden Place,2


In [107]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==3,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
238,India Basin,3


In [108]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==4,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
232,Forest Knolls,4


In [109]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==5,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
293,Sea Cliff,5


In [110]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==6,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
200,Ashbury Heights,6


In [111]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==7,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
210,Cayuga Terrace,7


### VISUALIZING THE TWO CITIES ON THE WORLD MAP

In [85]:
city_loca=[]
for city in ['Bengaluru, India','San Francisco, California']:
    address = city
    geolocator = Nominatim(user_agent='sunnypot1989@gmail.com')
    loca = geolocator.geocode(address)
    city_loca.append([city, loca.longitude, loca.latitude])
    time.sleep(2.0)
city_loca

[['Bengaluru, India', 77.5912997, 12.9791198],
 ['San Francisco, California', -122.4199061, 37.7790262]]

In [87]:
#add markers to the cities
map = folium.Map()
for i in city_loca:
    label,lng,lat = i[0],i[1],i[2]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
map

### CLUSTERING BANGLORE AND SAN FRANCISCO TOGETHER

In [88]:
df_onehot = onehot_df(df_venues)
print(df_onehot.shape)
df_onehot.head()

(6091, 397)


,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Andhra Restaurant,...,Vietnamese Restaurant,Vineyard,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df_grouped=df_onehot.groupby('Neighbourhood').mean().reset_index()
print(df_grouped.shape)
df_grouped.head()

(278, 397)


,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Andhra Restaurant,...,Vietnamese Restaurant,Vineyard,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,A Narayanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,Agaram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,Agrahara Dasarahalli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Alamo Square,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.013514


In [90]:
df_venues_sorted = sorted_dataframe(df_grouped)
df_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,A Narayanapura,Fast Food Restaurant,Pharmacy,Dessert Shop,Bus Stop,Pizza Place,Yoga Studio,Flower Shop,Filipino Restaurant,Film Studio,...,Fish & Chips Shop,Fish Market,Flea Market,Food,Farm,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium
1,Adugodi,Playground,Café,Athletics & Sports,Ice Cream Shop,Bus Station,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,...,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio,Farm,Food & Drink Shop,Food Court,Food Stand,Food Truck
2,Agaram,Burger Joint,Bus Station,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,...,Flea Market,Flower Shop,Food,Food Court,Cultural Center,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant
3,Agrahara Dasarahalli,Indian Restaurant,Bagel Shop,Breakfast Spot,Bar,Café,Bus Station,Vegetarian / Vegan Restaurant,Bakery,Flower Shop,...,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio,Food,Food & Drink Shop,Food Stand
4,Alamo Square,Bar,Park,BBQ Joint,Café,Seafood Restaurant,Liquor Store,Sushi Restaurant,Pizza Place,Ethiopian Restaurant,...,Hotel,Record Shop,Coffee Shop,Convenience Store,Pet Store,Hunan Restaurant,Comic Shop,Yoga Studio,Cocktail Bar,Restaurant


In [91]:
#set number of clusters
kclusters = 8

df_grouped_clustering = df_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

278

**Note** : _df_sf is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [92]:
df.shape

(317, 6)

In [93]:
df_labels = df.sort_values(['Neighbourhood'])
x=[]
for i in df_labels['Neighbourhood']:
    if i in df_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_labels=df_labels[x]
print(df_labels.shape)

(278, 6)


In [94]:
df_labels['Cluster Labels'] = kmeans1.labels_
df_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
55,A Narayanapura,Bangalore,KA,India,12.993195,77.670377,2
146,Adugodi,Bangalore,KA,India,12.942847,77.610416,2
113,Agaram,Bangalore,KA,India,12.945704,77.637886,2
104,Agrahara Dasarahalli,Bangalore,KA,India,12.979063,77.540682,7
198,Alamo Square,San Francisco,CA,USA,37.776360,-122.434689,2


In [155]:
df_labels.sort_index(inplace=True)

In [156]:
df_final = pd.concat([df,df_labels],axis=0)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [157]:
df_final.groupby(['Cluster Labels','City']).count()


Country  Latitude  Longitude  Neighbourhood  \
Cluster Labels City                                                         
0.0            Bangalore            7         7          7              7   
               San Francisco        7         7          7              7   
1.0            Bangalore           15        15         15             15   
2.0            Bangalore           76        76         76             76   
               San Francisco      100       100        100            100   
3.0            Bangalore            1         1          1              1   
4.0            Bangalore            7         7          7              7   
5.0            Bangalore            2         2          2              2   
6.0            San Francisco        3         3          3              3   
7.0            Bangalore           59        59         59             59   
               San Francisco        1         1          1              1   

                              State  
Cluster Labels City                  
0.0            Bangalore          7  
               San Francisco      7  
1.0            Bangalore         15  
2.0            Bangalore         76  
               San Francisco    100  
3.0            Bangalore          1  
4.0            Bangalore          7  
5.0            Bangalore          2  
6.0            San Francisco      3  
7.0            Bangalore         59  
               San Francisco      1

In [158]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

map_city_clusters(df_labels,latitude,longitude)

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [159]:
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

map_city_clusters(df_labels,latitude,longitude)

The geograpical coordinate of San Francisco are 12.9791198, 77.5912997.


In [120]:
df_labels.loc[df_labels['Cluster Labels']==0,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
13,Bagalakunte,0
210,Cayuga Terrace,0
231,Forest Hill,0
234,Golden Gate Heights,0
144,Hombegowda Nagar,0
238,India Basin,0
169,Jayanagar East,0
100,Kamakshipalya,0
271,Mount Davidson,0
59,Sagayara Puram,0


In [121]:
df_labels.loc[df_labels['Cluster Labels']==1,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
133,Bapuji Nagar,1
174,Bommanahalli,1
6,Byatarayanapura,1
1,Chowdeswari Ward,1
156,Gali Anjenaya Temple Ward,1
132,Hampi Nagar,1
188,Honga Sandra,1
135,Jagajivanaram Nagar,1
45,Jayachamarajendra Nagar,1
88,Jogupalya,1


In [122]:
df_labels.loc[df_labels['Cluster Labels']==2,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
55,A Narayanapura,2
146,Adugodi,2
113,Agaram,2
198,Alamo Square,2
195,Anjanapura,2
199,Anza Vista,2
192,Arakere,2
200,Ashbury Heights,2
140,Azad Nagar,2
201,Balboa Park,2


In [123]:
df_labels.loc[df_labels['Cluster Labels']==3,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
53,Hudi,3


In [124]:
df_labels.loc[df_labels['Cluster Labels']==4,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
72,Kottegepalya,4
124,Marenahalli,4
47,Muneshwara Nagar,4
134,Padarayanapura,4
69,Rajagopal Nagar,4
136,Rayapuram,4
50,Vijnanapura,4


In [125]:
df_labels.loc[df_labels['Cluster Labels']==5,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
119,Cottonpete,5
167,Pattabhi Ram Nagar,5


In [126]:
df_labels.loc[df_labels['Cluster Labels']==6,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
214,Clarendon Heights,6
251,Laurel Heights,6
270,Monterey Heights,6


In [127]:
df_labels.loc[df_labels['Cluster Labels']==7,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
104,Agrahara Dasarahalli,7
34,Aramane Nagara,7
2,Atturu,7
175,BTM Layout,7
26,Banasavadi,7
153,Basavanagudi,7
149,Bellanduru,7
187,Bilekhalli,7
168,Byra Sandra,7
56,C.V. Raman Nagar,7
